In [1]:
import numpy as np
import requests
import json
from requests.auth import HTTPBasicAuth
import matplotlib.pyplot as plt

import folium
from folium import plugins
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import sys
sys.path.insert(1, '../')

import apikey
from get_data_API import get_data

# In this notebook
Testing how to use dataframe cleanly with less lines of code and making code more readable

# Get sensor locations from old API

In [2]:
# Set the API endpoint and request all devices
url = 'https://smartcampus.oulu.fi/manage/api/devices/listAll'
old_sensor_data = []

# Send the API request
response = requests.get(url)

# Check the response status code
if response.status_code == 200:
    print("200")
else:
    print("Error: API request failed with status code", response.status_code)
    
for line in response.iter_lines(decode_unicode=True):
    old_sensor_data.append(json.loads(line))

200


In [3]:
#First element on the list
print(old_sensor_data[0][0]['deviceId'])
print(old_sensor_data[0][0]['location']['coordinates'])

A81758FFFE046433
[65.05765, 25.46897]


In [4]:
#Dictionary with key and coordinates
sensor_list = {}
for sensor in old_sensor_data[0]:
    
    #Check for Nones in data
    if sensor['location'] == None:
        continue
        
    #without hyphens; ex. A81758FFFE046433
    old_id = sensor['deviceId']
    #with hyphens; ex. A8-17-58-FF-FE-04-64-33
    new_id = '-'.join(old_id[i:i+2] for i in range(0, len(old_id), 2))
    
    #Create dictionary {key new_id : value coordinates}
    sensor_list[new_id] = sensor['location']['coordinates']
    
    
    
sensor_list

{'A8-17-58-FF-FE-04-64-33': [65.05765, 25.46897],
 'A8-17-58-FF-FE-03-0F-6A': [65.05875, 25.46565],
 'A8-17-58-FF-FE-03-10-18': [65.05884, 25.46823],
 'A8-17-58-FF-FE-04-65-00': [65.06081, 25.46679],
 'A8-17-58-FF-FE-03-10-09': [65.05842, 25.46385],
 'A8-17-58-FF-FE-03-10-87': [65.0585, 25.46674],
 'A8-17-58-FF-FE-04-64-F8': [65.05994, 25.46657],
 'A8-17-58-FF-FE-03-0F-8B': [65.05961, 25.466],
 'A8-17-58-FF-FE-03-0F-68': [65.05963, 25.46852],
 'A8-17-58-FF-FE-03-10-0D': [65.05998, 25.46886],
 'A8-17-58-FF-FE-03-0F-93': [65.05965, 25.46385],
 'A8-17-58-FF-FE-04-64-AC': [65.05688, 25.46774],
 'A8-17-58-FF-FE-03-10-90': [65.05807, 25.46889],
 'A8-17-58-FF-FE-03-10-0F': [65.05785, 25.46883],
 'A8-17-58-FF-FE-04-64-B3': [65.05776, 25.46879],
 'A8-17-58-FF-FE-03-10-33': [65.05719, 25.46788],
 'A8-17-58-FF-FE-04-63-38': [65.05967, 25.4634],
 'A8-17-58-FF-FE-03-10-20': [65.05988, 25.46846],
 'A8-17-58-FF-FE-03-0F-D4': [65.06106, 25.46617],
 'A8-17-58-FF-FE-03-0F-B2': [65.06113, 25.46619],
 'A8

# Get sensor data from new API

In [5]:
#smart campus api key
headers = {
    'Content-type': 'application/json', 
    'Authorization': f'{apikey.APIKEY}',
}

In [6]:
# Set the API endpoint and request 15000 datapoints
url = 'https://query-api.rahtiapp.fi/events?limit=15000'

# Send the API request
response = requests.get(url, headers=headers)

# Check the response status code
if response.status_code == 200:
    # Print the sensor data
    sensor_data = response.iter_lines()
else:
    print("Error: API request failed with status code", response.status_code)

In [7]:
#Create list out of response
resp = []
for line in response.iter_lines(decode_unicode=True):
    resp.append(json.loads(line))

In [8]:
#Singe data point
resp[0][0]

{'time': '2023-04-12T00:00:06.578Z',
 'temperature': 15.5,
 'humidity': 93,
 'light': None,
 'motion': 0,
 'co2': None,
 'battery': 3.15,
 'deveui': 'a8-17-58-ff-fe-04-6f-2a',
 'deveui_1': 'a8-17-58-ff-fe-04-6f-2a'}

In [9]:
try:
    sensor_list[resp[0][0]['deveui'].upper()]
    print("Sensor found")
except KeyError:
    print("No locations for sensor")

Sensor found


In [10]:
#Create dataframe
df = pd.DataFrame(columns=['ID','time','temperature','humidity','light','motion','co2', 'coordinates'])

In [11]:
#iterate trough response list
for measurement in resp[0]:
    
    #add row to dataframe
    df.loc[len(df.index)]=[measurement['deveui'],       #id
                           measurement['time'], #timestamp
                           measurement['temperature'],  #temperature
                           measurement['humidity'],     #humidity
                           measurement['light'],        #illuminance
                           measurement['motion'],       #motion
                           measurement['co2'],          #CO2
                           sensor_list[measurement['deveui'].upper()] #coordinates[lat, lng]
                          ]

In [12]:
for measurement in resp[0]:
    print(sensor_list[measurement['deveui'].upper()])

[65.06345, 25.46497]
[65.05895, 25.46782]
[65.05902, 25.46461]
[65.05705, 25.46767]
[65.06132, 25.46819]
[65.06107, 25.46675]
[65.05804357640024, 25.466386377811435]
[65.0611, 25.46679]
[65.05813404631877, 25.466177165508274]
[65.05965, 25.46339]
[65.05786, 25.46883]
[65.05782, 25.46833]
[65.05791, 25.46887]
[65.06036, 25.4667]
[65.05769, 25.4676]
[65.05864, 25.46656]
[65.0606, 25.4667]
[65.05877, 25.46772]
[65.06346, 25.46512]
[65.05844, 25.46344]
[65.05891, 25.46803]
[65.06131, 25.46819]
[65.05813404631877, 25.466177165508274]
[65.0586, 25.46627]
[65.05991, 25.46633]
[65.05755, 25.46833]
[65.05875, 25.46565]
[65.06341, 25.46533]
[65.05808, 25.4689]
[65.0585, 25.46674]
[65.0586, 25.46656]
[65.05891, 25.46798]
[65.05834, 25.46676]
[65.05776, 25.46895]
[65.05877, 25.46641]
[65.05995, 25.46633]
[65.05875, 25.46772]
[65.05792, 25.46972]
[65.05745, 25.46883]
[65.06043, 25.46667]
[65.05813404631877, 25.466177165508274]
[65.05855, 25.46654]
[65.05797, 25.46894]
[65.05953, 25.46647]
[65.05744

In [13]:
df

,ID,time,temperature,humidity,light,motion,co2,coordinates
0,a8-17-58-ff-fe-04-6f-2a,2023-04-12T00:00:06.578Z,15.5,93,None,0,None,"[65.06345, 25.46497]"
1,a8-17-58-ff-fe-04-65-30,2023-04-12T00:00:08.213Z,19.6,22,2,88,None,"[65.05895, 25.46782]"
2,a8-17-58-ff-fe-03-10-1a,2023-04-12T00:00:10.033Z,21.0,21,0,0,433,"[65.05902, 25.46461]"
3,a8-17-58-ff-fe-03-10-51,2023-04-12T00:00:11.395Z,22.2,24,0,0,653,"[65.05705, 25.46767]"
4,a8-17-58-ff-fe-04-65-2f,2023-04-12T00:00:11.578Z,21.1,21,0,79,None,"[65.06132, 25.46819]"
...,...,...,...,...,...,...,...,...
12644,a8-17-58-ff-fe-03-10-43,2023-04-12T11:49:51.289Z,19.1,27,2084,0,446,"[65.05757, 25.46833]"
12645,a8-17-58-ff-fe-03-0f-b6,2023-04-12T11:49:51.708Z,21.3,22,227,10,514,"[65.06133, 25.46981]"
12646,a8-17-58-ff-fe-04-63-3f,2023-04-12T11:49:52.945Z,19.6,25,73,78,None,"[65.05906, 25.46688]"
12647,a8-17-58-ff-fe-03-0f-65,2023-04-12T11:49:56.324Z,20.8,24,120,2,655,"[65.05776, 25.46891]"


# Format data to fit HeatMap functions parameters

In [14]:
''' 
HeatMapWithTime()

Parameters
    ----------
    data: list of list of points of the form [lat, lng] or [lat, lng, weight]
        The points you want to plot. The outer list corresponds to the various time
        steps in sequential order. (weight is in (0, 1] range and defaults to 1 if
        not specified for a point)
    index: Index giving the label (or timestamp) of the elements of data. Should have
        the same length as data, or is replaced by a simple count if not specified.
    name : string, default None
        The name of the Layer, as it will appear in LayerControls.
'''

#Min-Max normalize column to get weights between [0, 1] range
column = "motion"
df[column] = (df[column] - df[column].min()) / (df[column].max() - df[column].min())
column = "light"
df[column] = (df[column] - df[column].min()) / (df[column].max() - df[column].min())



In [15]:
motion_data = df[["coordinates", "motion", "time"]]

In [16]:
data = []
timestamps = []

for _, row in motion_data.iterrows():
    data.append([row['coordinates'][0],row['coordinates'][1],row['motion']])
    timestamps.append(row['time'])

data

[[65.06345, 25.46497, 0.0],
 [65.05895, 25.46782, 0.5269461077844312],
 [65.05902, 25.46461, 0.0],
 [65.05705, 25.46767, 0.0],
 [65.06132, 25.46819, 0.47305389221556887],
 [65.06107, 25.46675, 0.0],
 [65.05804357640024, 25.466386377811435, 0.0],
 [65.0611, 25.46679, 0.0],
 [65.05813404631877, 25.466177165508274, 0.0],
 [65.05965, 25.46339, 0.0],
 [65.05786, 25.46883, 0.011976047904191617],
 [65.05782, 25.46833, 0.0],
 [65.05791, 25.46887, 0.0],
 [65.06036, 25.4667, 0.0],
 [65.05769, 25.4676, 0.0],
 [65.05864, 25.46656, 0.0],
 [65.0606, 25.4667, 0.005988023952095809],
 [65.05877, 25.46772, 0.5149700598802395],
 [65.06346, 25.46512, 0.0],
 [65.05844, 25.46344, 0.0],
 [65.05891, 25.46803, 0.011976047904191617],
 [65.06131, 25.46819, 0.005988023952095809],
 [65.05813404631877, 25.466177165508274, 0.0],
 [65.0586, 25.46627, 0.011976047904191617],
 [65.05991, 25.46633, 0.5029940119760479],
 [65.05755, 25.46833, 0.0],
 [65.05875, 25.46565, 0.0],
 [65.06341, 25.46533, nan],
 [65.05808, 25.4689

# Change time to 24h format

In [17]:
timestamps[-1]

'2023-04-12T11:50:02.608Z'

In [18]:
import datetime
import numpy
from datetime import timedelta

In [19]:
test = datetime.datetime.fromisoformat(timestamps[-1])

In [20]:
test.time()

datetime.time(11, 50, 2, 608000)

In [21]:
datetime.time(7, 45, 0, 0) < test.time()

True

In [22]:
timesOrganized = []
measOrganized = []
measTemp = []
timesTemp = []
slot = None
oldSlot = None
index = 0

def groupData():
    global timesOrganized, measOrganized, measTemp, timesTemp, index, oldSlot, slot
    
    if slot == oldSlot: #Same as previous
        timesTemp.append(time)
        if not numpy.isnan(data[index][2]) and data[index][2] != 0.0:
            measTemp.append(data[index])
    elif oldSlot == None: #First value
        oldSlot = slot
        timesTemp.append(time)
        if not numpy.isnan(data[index][2]) and data[index][2] != 0.0:
            measTemp.append(data[index])
    else: #different as previous
        timesOrganized.append(timesTemp)
        measOrganized.append(measTemp)
        oldSlot = slot
        timesTemp = []
        measTemp = []
        timesTemp.append(time)
        if not numpy.isnan(data[index][2]) and data[index][2] != 0.0:
            measTemp.append(data[index])

for time in timestamps:
    minutes = datetime.datetime.fromisoformat(time).minute
    if minutes >= 0 and minutes < 15 :
        slot = 0
        groupData()
    if minutes >= 15 and minutes < 30 :
        slot = 1
        groupData()
    if minutes >= 30 and minutes < 45 :
        slot = 2
        groupData()
    if minutes >= 45 and minutes < 60 :
        slot = 3
        groupData()
    index = index + 1

In [23]:
i = 0
timeIndexes = []
for i in range(len(timesOrganized)):
    time = datetime.datetime.fromisoformat(timesOrganized[i][-1])
    time = time + timedelta(minutes=1)  
    time = time.replace(second=0, microsecond=0)
    timeIndexes.append(time)

In [24]:
printable = []
for time in timeIndexes:
    printable.append(time.strftime("%m/%d/%Y, %H:%M:%S"))
    
print(printable)
len(printable)


['04/12/2023, 00:15:00', '04/12/2023, 00:30:00', '04/12/2023, 00:45:00', '04/12/2023, 01:00:00', '04/12/2023, 01:15:00', '04/12/2023, 01:30:00', '04/12/2023, 01:45:00', '04/12/2023, 02:00:00', '04/12/2023, 02:15:00', '04/12/2023, 02:30:00', '04/12/2023, 02:45:00', '04/12/2023, 03:00:00', '04/12/2023, 03:15:00', '04/12/2023, 03:30:00', '04/12/2023, 03:45:00', '04/12/2023, 04:00:00', '04/12/2023, 04:15:00', '04/12/2023, 04:30:00', '04/12/2023, 04:45:00', '04/12/2023, 05:00:00', '04/12/2023, 05:15:00', '04/12/2023, 05:30:00', '04/12/2023, 05:45:00', '04/12/2023, 06:00:00', '04/12/2023, 06:15:00', '04/12/2023, 06:30:00', '04/12/2023, 06:45:00', '04/12/2023, 07:00:00', '04/12/2023, 07:15:00', '04/12/2023, 07:30:00', '04/12/2023, 07:45:00', '04/12/2023, 08:00:00', '04/12/2023, 08:15:00', '04/12/2023, 08:30:00', '04/12/2023, 08:45:00', '04/12/2023, 09:00:00', '04/12/2023, 09:15:00', '04/12/2023, 09:30:00', '04/12/2023, 09:45:00', '04/12/2023, 10:00:00', '04/12/2023, 10:15:00', '04/12/2023, 10

47

In [25]:
print(type(measOrganized[0]))
len(measOrganized)

<class 'list'>


47

In [26]:

m = folium.Map([65.06, 25.467], zoom_start=14.5)

#hm = plugins.HeatMapWithTime(measOrganized[1], auto_play=True, max_opacity=0.3)

m.add_child(plugins.HeatMapWithTime(measOrganized, index=printable, auto_play=True, max_opacity=0.3))

#hm.add_to(m)

m